In [4]:
pd.read_parquet('logs/03_osn-flight_table-overflights-etl-log.parquet')

,months
0,2022-01-01
1,2022-02-01
2,2022-03-01
3,2022-04-01
4,2022-05-01
5,2022-06-01
6,2022-07-01
7,2022-08-01
8,2022-09-01
9,2022-10-01


In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    lit, lag, when, to_date, concat, avg, abs, col, unix_timestamp, to_timestamp
)
from pyspark.sql import DataFrame
import os
import pandas as pd
from datetime import datetime, date
import dateutil.relativedelta
import calendar
import h3_pyspark
import gc

# Settings
project = "project_opdi"
test_day = date(2024, 1, 1)

# Spark Session Initialization
spark = SparkSession.builder \
    .appName("OPDI Ingestion") \
    .config("spark.hadoop.fs.azure.ext.cab.required.group", "eur-app-opdi") \
    .config("spark.kerberos.access.hadoopFileSystems", "abfs://storage-fs@cdpdllive.dfs.core.windows.net/data/project/opdi.db/unmanaged") \
    .config("spark.executor.extraClassPath", "/opt/spark/optional-lib/iceberg-spark-runtime-3.3_2.12-1.3.1.1.20.7216.0-70.jar") \
    .config("spark.driver.extraClassPath", "/opt/spark/optional-lib/iceberg-spark-runtime-3.3_2.12-1.3.1.1.20.7216.0-70.jar") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "abfs://storage-fs@cdpdllive.dfs.core.windows.net/data/project/opdi.db/unmanaged") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "5G") \
    .config("spark.executor.memory", "16G") \
    .config("spark.executor.memoryOverhead", "3G") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "3") \
    .config("spark.dynamicAllocation.maxExecutors", "4") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "400s") \
    .config("spark.driver.maxResultSize", "6g") \
    .config("spark.shuffle.compress", "true") \
    .config("spark.shuffle.spill.compress", "true") \
    .enableHiveSupport() \
    .getOrCreate()


Setting spark.hadoop.yarn.resourcemanager.principal to quinten.goens


In [3]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from traffic.core import Traffic

df = spark.sql(
    f"SELECT * FROM {project}.osn_tracks "
    "WHERE event_time >= TIMESTAMP '2024-01-01 12:00:00' "
    "AND event_time < TIMESTAMP '2024-01-01 14:00:00'"
)

# Define a Pandas UDF
@pandas_udf("string", PandasUDFType.GROUPED_MAP)
def process_trajectory(trajectory_df: pd.DataFrame) -> pd.DataFrame:
    traffic_obj = Traffic.from_dataframe(trajectory_df)
    
    # Example: Add a new column with summary results
    trajectory_df["traffic_summary"] = traffic_obj.summary()  # Modify as needed

    return trajectory_df  # Return the modified DataFrame

# Apply the function using groupBy (e.g., by flight ID)
result_df = spark_df.groupBy("flight_id").apply(process_trajectory)


+-------------------+------+------------------+------------------+------------------+------------------+-------------------+--------+---------+-----+-----+------+------------------+------------------+------------------+------------------+----------------+----------------+-------------+--------------------+---------------+---------------+-------------------+----------------------+
|         event_time|icao24|               lat|               lon|          velocity|           heading|          vert_rate|callsign|on_ground|alert|  spi|squawk|     baro_altitude|   baro_altitude_c|      geo_altitude|    geo_altitude_c| last_pos_update|    last_contact|      serials|            track_id|      h3_res_12|       h3_res_7|segment_distance_nm|cumulative_distance_nm|
+-------------------+------+------------------+------------------+------------------+------------------+-------------------+--------+---------+-----+-----+------+------------------+------------------+------------------+---------------